In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import sys
import seaborn as sns
sys.path.append('../src/')
sys.path.append('../src/exp_004')
from preprocess import read_data, get_data

In [3]:
train, test, submit = read_data()

In [4]:
train.head()

,id,year,month,day,Country,City,lat,lon,co_cnt,co_min,...,ws_min,ws_mid,ws_max,ws_var,dew_cnt,dew_min,dew_mid,dew_max,dew_var,pm25_mid
0,1,2019,1,1,Australia,Brisbane,-27.46794,153.02809,38,0.749,...,0.241,1.088,3.101,1.983,17,7.671,10.358,15.112,13.424,19.901
1,2,2019,1,1,Australia,Darwin,-12.46113,130.84185,47,2.594,...,0.828,3.473,7.396,10.411,62,21.324,23.813,24.221,2.021,13.741
2,3,2019,1,1,Australia,Melbourne,-37.81400,144.96332,17,1.190,...,0.000,2.107,8.089,15.719,22,10.309,13.133,15.422,6.355,25.918
3,4,2019,1,1,Australia,Newcastle,-32.92953,151.78010,63,4.586,...,0.284,0.503,3.592,2.485,116,7.146,10.685,13.344,9.417,174.370
4,5,2019,1,1,Australia,Perth,-31.95224,115.86140,47,4.689,...,0.500,0.755,3.396,1.937,93,1.091,3.277,12.272,4.109,167.063


In [5]:
train['test'] = 0
test['test'] = 1
all_data = pd.concat([train, test])

In [6]:
all_data['date'] = pd.to_datetime(all_data['year'].astype(str) + '-' + all_data['month'].astype(str) + '-' + all_data['day'].astype(str))
all_data.head()

,id,year,month,day,Country,City,lat,lon,co_cnt,co_min,...,ws_max,ws_var,dew_cnt,dew_min,dew_mid,dew_max,dew_var,pm25_mid,test,date
0,1,2019,1,1,Australia,Brisbane,-27.46794,153.02809,38,0.749,...,3.101,1.983,17,7.671,10.358,15.112,13.424,19.901,0,2019-01-01
1,2,2019,1,1,Australia,Darwin,-12.46113,130.84185,47,2.594,...,7.396,10.411,62,21.324,23.813,24.221,2.021,13.741,0,2019-01-01
2,3,2019,1,1,Australia,Melbourne,-37.81400,144.96332,17,1.190,...,8.089,15.719,22,10.309,13.133,15.422,6.355,25.918,0,2019-01-01
3,4,2019,1,1,Australia,Newcastle,-32.92953,151.78010,63,4.586,...,3.592,2.485,116,7.146,10.685,13.344,9.417,174.370,0,2019-01-01
4,5,2019,1,1,Australia,Perth,-31.95224,115.86140,47,4.689,...,3.396,1.937,93,1.091,3.277,12.272,4.109,167.063,0,2019-01-01


In [18]:
country_group = all_data.groupby('Country')['date'].agg(pd.Series.nunique).sort_values()
country_group

Country
Croatia                    530
Iran                       959
Netherlands                966
Bosnia and Herzegovina    1013
Vietnam                   1049
Russia                    1054
Israel                    1056
South Africa              1070
Canada                    1072
Germany                   1074
Serbia                    1077
Chile                     1082
Belgium                   1084
Mexico                    1085
Brazil                    1087
United Kingdom            1088
France                    1088
Thailand                  1089
Australia                 1091
Italy                     1091
India                     1092
Hungary                   1093
Turkey                    1093
Poland                    1093
United States             1094
Spain                     1094
Japan                     1095
South Korea               1095
Taiwan                    1095
China                     1095
Name: date, dtype: int64

In [20]:
japan = all_data[all_data['Country'] == 'Japan']
japan.date.nunique() / 3


365.0

学習データ→その地点での前後の日のpm25
テストデータ→その地点付近での前後の日のpm25

testデータの都市でtrainデータの中から一番近い都市をとりあえず探そう